In [1]:
import pandas as pd
from operator import itemgetter
from pprint import pprint
from datetime import datetime

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langsmith import Client

import os
os.environ["LANGCHAIN_PROJECT"] = "Agents with Langchain"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

client = Client()

In [3]:
News_api_key = os.environ["NEWS_API_KEY"]
Financial_news_api_key=os.environ["ALPHAVANTAGE_API_KEY"]

In [4]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_fireworks.chat_models import ChatFireworks
from langchain_core.output_parsers.string import StrOutputParser

In [195]:
TEMPERATURE = 0.02
MAX_NEW_TOKENS = 4000

MODELS = {
    "llama-2": ChatFireworks(
        model_name="accounts/fireworks/models/llama-v2-70b-chat",
        temperature=TEMPERATURE,
        max_tokens=MAX_NEW_TOKENS,            
    ),
    "llama-3": ChatFireworks(
        model_name="accounts/fireworks/models/llama-v3-70b-instruct",
        temperature=TEMPERATURE,
        max_tokens=MAX_NEW_TOKENS,            
    ),    
    "firefunction": ChatFireworks(
        model_name="accounts/fireworks/models/firefunction-v1",
        temperature=TEMPERATURE,
        max_tokens=MAX_NEW_TOKENS,            
    ),
    "gpt3_5turbo": ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=TEMPERATURE,
        max_tokens=MAX_NEW_TOKENS,
    ),    
    "gpt4o": ChatOpenAI(
        model="gpt-4o",
        temperature=TEMPERATURE,
        max_tokens=MAX_NEW_TOKENS,
    ),
    "mistral": ChatMistralAI(
        model="mistral-large-latest",
        temperature=TEMPERATURE,
        max_tokens=MAX_NEW_TOKENS,
    ),
    "gemini_flash": ChatGoogleGenerativeAI(
        model="gemini-1.5-flash-latest",
        temperature=TEMPERATURE,
        max_tokens=MAX_NEW_TOKENS,
    ),
    "gemini_pro": ChatGoogleGenerativeAI(
        model="gemini-1.5-pro-latest",
        temperature=TEMPERATURE,
        max_tokens=MAX_NEW_TOKENS,
    ),
}

router_llm = MODELS["gpt4o"]
tool_llm = MODELS["gpt4o"]

# Model output -> string
parser = StrOutputParser()

# Agents with vendor specific function calling APIs

Let's create a new agent, that uses the vendors implementation of the function calling api.

Like OpenAI functions

Also take a look at the llms and vendors that support function calling apis [here](https://python.langchain.com/docs/integrations/chat/?ref=blog.langchain.dev)

In [196]:
#from langchain.tools import WikipediaQueryRun

from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
#from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun
from langchain_community.tools.pubmed.tool import PubmedQueryRun

from langchain_community.utilities.alpha_vantage import AlphaVantageAPIWrapper
from langchain.agents import create_tool_calling_agent, AgentExecutor, load_tools
from langchain_core.prompts import ChatPromptTemplate
from langchain.globals import set_debug, set_verbose
from langchain_core.output_parsers.string import StrOutputParser

set_debug(False)
set_verbose(False)

In [197]:
DATE = datetime.today().strftime('%Y-%m-%d')
#prompt = f"Create a table that contains the date as well as the maximum, minimum and avarage temperature values separately for each of 2024-05-18, 2024-05-19 and 2024-05-20 in Budapest."
prompt = f"Create a table that contains the date as well as the maximum, minimum and avarage temperature values as well as the sum of precipitations separately for each of the coming 7 days in Budapest"
print(prompt)

#pubmed_tool = PubmedQueryRun()

#llm_math_tool, wikipedia_tool, weather_tool, news_tool = load_tools(["llm-math","wikipedia", "open-meteo-api", "news-api"], news_api_key=News_api_key, llm=llm)
llm_math_tool, websearch_tool, wikipedia_tool, arxiv_tool, weather_tool, news_tool = load_tools(["llm-math","ddg-search", "wikipedia", "arxiv", "open-meteo-api", "news-api"], news_api_key=News_api_key, llm=tool_llm)

#llm_math_tool.invoke("What is the square root of 4?")
#wikipedia_tool.invoke("How many people live in Prague?")
#print()
#print(weather_tool.invoke(prompt, verbose=True))
#print("\n\n")
#print(news_tool.invoke("What are the 10 most important news in Prague? Answer in 10 bullet points"))
#print(arxiv_tool.invoke("List the title of 10 scientific papers about LLM agents published in this year.", verbose=True))
#print(websearch_tool.invoke("Who won the most Oscar in this year?"))
#print(pubmed_tool.invoke("List papers about Vortioxetin."))

Create a table that contains the date as well as the maximum, minimum and avarage temperature values as well as the sum of precipitations separately for each of the coming 7 days in Budapest


In [198]:
"""Callback Handler that writes to a variable."""


from __future__ import annotations

from typing import TYPE_CHECKING, Any, Dict, Optional

from langchain_core.callbacks.base import BaseCallbackHandler
#from langchain_core.utils import print_text

if TYPE_CHECKING:
    from langchain_core.agents import AgentAction, AgentFinish


_TEXT_COLOR_MAPPING = {
    "blue": "36;1",
    "yellow": "33;1",
    "pink": "38;5;200",
    "green": "32;1",
    "red": "31;1",
}

def get_color_mapping(
    items: List[str], excluded_colors: Optional[List] = None
) -> Dict[str, str]:
    """Get mapping for items to a support color."""
    colors = list(_TEXT_COLOR_MAPPING.keys())
    if excluded_colors is not None:
        colors = [c for c in colors if c not in excluded_colors]
    color_mapping = {item: colors[i % len(colors)] for i, item in enumerate(items)}
    return color_mapping


def get_colored_text(text: str, color: str) -> str:
    """Get colored text."""
    color_str = _TEXT_COLOR_MAPPING[color]
    return f"\u001b[{color_str}m\033[1;3m{text}\u001b[0m"


def get_bolded_text(text: str) -> str:
    """Get bolded text."""
    return f"\033[1m{text}\033[0m"


def print_text(
    text: str, color: Optional[str] = None, end: str = ""
) -> str:
    """Print text with highlighting and no end characters."""
    #print(f"color = {color}")
    text_to_print = get_colored_text(text, color) if color else text
    #print(text_to_print, end=end)  # noqa: T201
    return text_to_print

class VariableCallbackHandler(BaseCallbackHandler):
    """Callback Handler that prints to a variable."""

    def __init__(self, color: Optional[str] = None) -> None:
        """Initialize callback handler."""
        self.color = "blue"

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs: Any
    ) -> None:
        """Print out that we are entering a chain."""
        class_name = serialized.get("name", serialized.get("id", ["<unknown>"])[-1])
        #print(f"\n\n\033[1m> Entering new {class_name} chain...\033[0m")  # noqa: T201
        trace_list.append(f"\033[1m> Entering new {class_name} chain...\033[0m")

    def on_chain_end(self, outputs: Dict[str, Any], **kwargs: Any) -> None:
        """Print out that we finished a chain."""
        #print("\n\033[1m> Finished chain.\033[0m")  # noqa: T201
        trace_list.append(f"\033[1m> Finished chain.\033[0m")
        
    def on_agent_action(
        self, action: AgentAction, color: Optional[str] = None, **kwargs: Any
    ) -> Any:
        """Run on agent action."""
        #print_text(action.log, color=color or self.color)
        new_text = print_text(action.log, color=color or self.color)
        trace_list.append(new_text)

    def on_tool_end(
        self,
        output: Any,
        color: Optional[str] = None,
        observation_prefix: Optional[str] = None,
        llm_prefix: Optional[str] = None,
        **kwargs: Any,
    ) -> None:
        """If not the final action, print out observation."""
        output = str(output)
        if observation_prefix is not None:
            #print_text(f"\n{observation_prefix}")
            trace_list.append(f"{observation_prefix}")
        if llm_prefix is not None:
            #print_text(f"\n{llm_prefix}")
            trace_list.append(f"{llm_prefix}")

    def on_text(
        self,
        text: str,
        color: Optional[str] = None,
        end: str = "",
        **kwargs: Any,
    ) -> None:
        """Run when agent ends."""
        #print_text(text, color=color or self.color, end=end)
        new_text = print_text(text, color=color or self.color, end=end)
        trace_list.append(new_text)

    def on_agent_finish(
        self, finish: AgentFinish, color: Optional[str] = None, **kwargs: Any
    ) -> None:
        """Run on agent end."""
        #print_text(finish.log, color=color or self.color, end="\n")
        new_text = print_text(finish.log, color=color or self.color, end="")
        trace_list.append(new_text)


In [199]:
#from langchain_core.callbacks import FileCallbackHandler
#filecallhandler = FileCallbackHandler(filename="callback.txt", mode="w", color="red")
trace_list = []
varcallhandler = VariableCallbackHandler()

#tools=[weather_tool]
tools=[llm_math_tool, arxiv_tool, news_tool, weather_tool, wikipedia_tool, websearch_tool] # More tools could be added
# Be careful with older tools, they might break with newer models

prompt = ChatPromptTemplate.from_messages([
    ("system", f"You're a helpful assistant. Always use tools to answer questions. Always use llm_math_tool for calculations, even when adding 2 numbers or calculating averages. The current date is {DATE}"),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])
agent = create_tool_calling_agent(router_llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, callbacks=[varcallhandler], verbose = False)
# , enable_automatic_function_calling=True

agent_chain = agent_executor | itemgetter("output")

In [206]:
prompt = "What are the maximum, minimum and average temperature values as well as the sum of precipitations on each of the coming 3 days in Budapest?"
#prompt = f"Create a table that contains the date as well as the maximum, minimum and average temperature values as well as the sum of precipitations in the coming 7 days in Budapest"
#prompt = f"""Create a table that contains the date as well as the maximum, minimum and average temperature values as well as the sum of precipitations separately for each of the coming 7 days in Budapest.
#Then include a line with each of the average of the maximum, minimum and avarage temperature values for these 7 days. And after all that also write a list with 10 current news in Budapest Hungary."""

trace_list = []
streaming = True

response = ""
if streaming == True:
    for new_token in agent_chain.stream({"input": prompt}):
        #if new_token is not None:
        print(new_token, end="")
        response = response + new_token
else:
    response = agent_chain.invoke({"input": prompt})
    print(response)
print("")

#response = agent_chain.stream({"input": prompt})

Here are the weather details for Budapest over the coming 7 days:

- **May 22, 2024**: 
  - Max Temp: 21.1°C
  - Min Temp: 16.1°C
  - Precipitation: 3.10 mm

- **May 23, 2024**: 
  - Max Temp: 24.5°C
  - Min Temp: 14.8°C
  - Precipitation: 7.30 mm

- **May 24, 2024**: 
  - Max Temp: 20.5°C
  - Min Temp: 16.5°C
  - Precipitation: 13.70 mm

- **May 25, 2024**: 
  - Max Temp: 24.9°C
  - Min Temp: 14.9°C
  - Precipitation: 0.30 mm

- **May 26, 2024**: 
  - Max Temp: 26.1°C
  - Min Temp: 16.5°C
  - Precipitation: 0.00 mm

- **May 27, 2024**: 
  - Max Temp: 27.3°C
  - Min Temp: 17.0°C
  - Precipitation: 0.00 mm

- **May 28, 2024**: 
  - Max Temp: 27.2°C
  - Min Temp: 16.5°C
  - Precipitation: 0.00 mm

### Summary:
- **Average Maximum Temperature**: 24.51°C
- **Average Minimum Temperature**: 16.04°C
- **Total Precipitation**: 24.40 mm


In [207]:
print(response)

Here are the weather details for Budapest over the coming 7 days:

- **May 22, 2024**: 
  - Max Temp: 21.1°C
  - Min Temp: 16.1°C
  - Precipitation: 3.10 mm

- **May 23, 2024**: 
  - Max Temp: 24.5°C
  - Min Temp: 14.8°C
  - Precipitation: 7.30 mm

- **May 24, 2024**: 
  - Max Temp: 20.5°C
  - Min Temp: 16.5°C
  - Precipitation: 13.70 mm

- **May 25, 2024**: 
  - Max Temp: 24.9°C
  - Min Temp: 14.9°C
  - Precipitation: 0.30 mm

- **May 26, 2024**: 
  - Max Temp: 26.1°C
  - Min Temp: 16.5°C
  - Precipitation: 0.00 mm

- **May 27, 2024**: 
  - Max Temp: 27.3°C
  - Min Temp: 17.0°C
  - Precipitation: 0.00 mm

- **May 28, 2024**: 
  - Max Temp: 27.2°C
  - Min Temp: 16.5°C
  - Precipitation: 0.00 mm

### Summary:
- **Average Maximum Temperature**: 24.51°C
- **Average Minimum Temperature**: 16.04°C
- **Total Precipitation**: 24.40 mm


In [208]:
for trace in trace_list:
    print(trace)

> Entering new AgentExecutor chain...

Invoking: `OpenMeteoAPI` with `What are the maximum, minimum and average temperature values as well as the sum of precipitations on each of the coming 7 days in Budapest?`




Invoking: `Calculator` with `(21.1 + 24.5 + 20.5 + 24.9 + 26.1 + 27.3 + 27.2) / 7`




Invoking: `Calculator` with `(16.1 + 14.8 + 16.5 + 14.9 + 16.5 + 17.0 + 16.5) / 7`




Invoking: `Calculator` with `3.10 + 7.30 + 13.70 + 0.30 + 0.00 + 0.00 + 0.00`



Here are the weather details for Budapest over the coming 7 days:

- **May 22, 2024**: 
  - Max Temp: 21.1°C
  - Min Temp: 16.1°C
  - Precipitation: 3.10 mm

- **May 23, 2024**: 
  - Max Temp: 24.5°C
  - Min Temp: 14.8°C
  - Precipitation: 7.30 mm

- **May 24, 2024**: 
  - Max Temp: 20.5°C
  - Min Temp: 16.5°C
  - Precipitation: 13.70 mm

- **May 25, 2024**: 
  - Max Temp: 24.9°C
  - Min Temp: 14.9°C
  - Precipitation: 0.30 mm

- **May 26, 2024**: 
  - Max Temp: 26.1°C
  - Min Temp: 16.5°C
  - Precipitation: 0.00 mm

- **May

In [ ]:
print(agent_chain.invoke({"input": "Which is more populous city, Budapest or Prague?"}))

In [ ]:
#agent_executor.invoke({"input": "How many people live in Budapest and Prague together?"})
print(agent_chain.invoke({"input": "How many people live in Budapest and Prague together?"}))

In [ ]:
print(agent_chain.invoke({"input": "What is the ratio of inhabitants in percentage in Prague vs Budapest?"}))

In [ ]:
print(agent_chain.invoke({"input": "What is the weather like in Budapest?"}))

In [ ]:
print(agent_chain.invoke({"input": "What is the weather like in Prague?"}))

In [ ]:
print(agent_chain.invoke({"input": "Where is the wether warmer currently, in Budapest or Prague?"}))

In [ ]:
print(agent_chain.invoke({"input": "What are the 10 trending news in Budapest? Answer in 10 bullet points."}))

In [ ]:
print(agent_chain.invoke({"input": "What are the 10 trending news in Prague? Answer in 10 bullet points."}))

In [ ]:
print(agent_chain.invoke({"input": "Who won the most Oscar in this year?"}))

In [ ]:
print(agent_chain.invoke({"input": "List the title of 10 scientific papers about LLM agents published in this year."}))